In [1]:
import pyspark
from pyspark.sql import types
from pyspark.sql import SparkSession
import findspark
import pandas as pd
import os

#findspark.init()

!spark-shell --version

print(pyspark.__file__)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.4
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.26
Branch HEAD
Compiled by user yangjie01 on 2024-12-17T04:51:46Z
Revision a6f220d951742f4074b37772485ee0ec7a774e7d
Url https://github.com/apache/spark
Type --help for more information.
/Users/koray/opt/anaconda3/lib/python3.9/site-packages/pyspark/__init__.py


In [2]:
os.chdir('/Users/koray/Documents/GitHub/deng25/05_spark')
os.getcwd()

# Create SparkSession
#local master - this master method is for cardinating clusters
#number of clusters to be used
spark = SparkSession.builder.master("local[3]") \
                    .appName('test-spark') \
                    .getOrCreate()

print(f'The PySpark {spark.version} version is running...')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 14:38:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The PySpark 3.5.4 version is running...


In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [10]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [9]:
df.printSchema() #In a parquet file, schema is already saved with the file. No need to define a schema like in a csv.

NameError: name 'df' is not defined

In [8]:
output_path = 'data/'
df = df.repartition(4)

NameError: name 'df' is not defined

In [6]:
df \
    .write.parquet('output')

In [5]:
df_p = spark.read.option('header','true').parquet('output/part-00000-e484fbd8-4557-43f7-a56b-873f809c40a7-c000.snappy.parquet')

In [11]:
df.registerTempTable('df_data')


/Users/koray/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [12]:
spark.sql("""
SELECT count(*) FROM df_data where date(tpep_pickup_datetime) = '2024-10-15' 
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [13]:
spark.sql("""
SELECT max(timestampdiff(hour,tpep_pickup_datetime,tpep_dropoff_datetime)) FROM df_data
""").show()

+---------------------------------------------------------------------+
|max(timestampdiff(hour, tpep_pickup_datetime, tpep_dropoff_datetime))|
+---------------------------------------------------------------------+
|                                                                  162|
+---------------------------------------------------------------------+



In [17]:
#http://localhost:4040

In [19]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-02 22:30:05--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2759:e200:b:20a5:b140:21, 2600:9000:2759:2c00:b:20a5:b140:21, 2600:9000:2759:7c00:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2759:e200:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: 'taxi_zone_lookup.csv'

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-02 22:30:05 (1.64 GB/s) - 'taxi_zone_lookup.csv' saved [12331/12331]



In [14]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [15]:
df_zones.registerTempTable('zones')


In [16]:
spark.sql("""
SELECT z.zone,count(*) FROM df_data d
left join zones z
on d.PULocationID = z.LocationID
group by 1
order by 2 ASC
""").show(truncate=False)

+---------------------------------------------+--------+
|zone                                         |count(1)|
+---------------------------------------------+--------+
|Governor's Island/Ellis Island/Liberty Island|1       |
|Rikers Island                                |2       |
|Arden Heights                                |2       |
|Green-Wood Cemetery                          |3       |
|Jamaica Bay                                  |3       |
|Charleston/Tottenville                       |4       |
|Rossville/Woodrow                            |4       |
|West Brighton                                |4       |
|Eltingville/Annadale/Prince's Bay            |4       |
|Port Richmond                                |4       |
|Great Kills                                  |6       |
|Crotona Park                                 |6       |
|Heartland Village/Todt Hill                  |7       |
|Mariners Harbor                              |7       |
|Oakwood                       